In [1]:
import csv
import io
import os
import re
import pandas as pd
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/colab_data'

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Mounted at /content/drive


In [2]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 356Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.39Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 289Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:14, 33.7Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 655Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.32Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.37Mit/s]


In [3]:
dataset = pd.read_csv(os.path.join(path, "taylor_swift_lyrics.csv"), encoding= 'unicode_escape')

In [4]:
def processFirstLine(lyrics, songID, songName, row):
    lyrics.append(row['lyric'] + '.\n')
    songID.append( row['year']*100+ row['track_n'])
    songName.append(row['track_title'])
    return lyrics,songID,songName
# define empty lists for the lyrics , songID , songName 
lyrics = []
songID = []
songName = []

# songNumber indicates the song number in the dataset
songNumber = 1

# i indicates the song number
i = 0
isFirstLine = True

# Iterate through every lyrics line and join them together for each song independently 
for index,row in dataset.iterrows():
    if(songNumber == row['track_n']):
        if (isFirstLine):
            lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
            isFirstLine = False
        else :
            #if we still in the same song , keep joining the lyrics lines    
            lyrics[i] +=  row['lyric'] + '.\n'
    #When it's done joining a song's lyrics lines , go to the next song :    
    else :
        lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
        songNumber = row['track_n']
        i+=1

# Define a new pandas DataFrame to save songID , songName , Lyrics in it to use them later
lyrics_data = pd.DataFrame({'songID':songID, 'songName':songName, 'lyrics':lyrics })
lyrics_data.head(3)

,songID,songName,lyrics
0,200601,Tim McGraw,He said the way my blue eyes shined.\nPut thos...
1,200602,Picture To Burn,"State the obvious, I didn't get my perfect fan..."
2,200603,Teardrops On My Guitar,Drew looks at me.\nI fake a smile so he won't ...


In [10]:
with open('/content/drive/My Drive/lyrics.txt', 'w') as f:
    for item in lyrics_data.lyrics.to_list():
        f.write("%s\n" % item)

In [11]:
gpt2.copy_file_from_gdrive('lyrics.txt')

In [13]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset='lyrics.txt',
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.93it/s]


dataset has 51982 tokens
Training...
[10 | 28.66] loss=1.83 avg=1.83
[20 | 50.18] loss=1.46 avg=1.65
[30 | 72.17] loss=1.71 avg=1.67
[40 | 94.68] loss=1.47 avg=1.62
[50 | 117.74] loss=1.31 avg=1.55
[60 | 140.54] loss=1.23 avg=1.50
[70 | 163.15] loss=1.26 avg=1.46
[80 | 185.83] loss=0.77 avg=1.37
[90 | 208.63] loss=0.87 avg=1.31
[100 | 231.39] loss=0.52 avg=1.23
[110 | 254.08] loss=0.36 avg=1.15
[120 | 276.77] loss=0.37 avg=1.08
[130 | 299.51] loss=0.16 avg=1.00
[140 | 322.25] loss=0.22 avg=0.94
[150 | 345.00] loss=0.14 avg=0.89
[160 | 367.76] loss=0.12 avg=0.84
[170 | 390.51] loss=0.09 avg=0.79
[180 | 413.27] loss=0.08 avg=0.75
[190 | 436.01] loss=0.09 avg=0.71
[200 | 458.76] loss=0.08 avg=0.67
======== SAMPLE 1 ========
 what is it that makes you want to go sit in your chair by the window?.
Yes, and I want to break your heart in the same way that you fell in love?.
You say it's simple, I'll show you.
That it's not the first time, it's not the last time.
It's not the last moment, it's 

In [14]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="My Love",
              nsamples=5,
              batch_size=5
              )

My Love is a secret I got, and I'm yours to keep.
And I'm so chill your a mystery.
But my love is a secret I got, and I'm yours to keep.
And I'm so chill your a mystery.
But my love is a secret I got.
And I'm yours to keep.
Baby, I'm scared to see the end.
Of all the years that we spent together.
And he never saw me coming.
And asking me to listen 'round is when I get scared.
So I play my own game.
But you know you better than I do.
Baby, I'm scared to see the end.
Of all the years that we spent together.
And he never saw me coming.
And asking me to listen 'round is when I get scared.
So I play my own game.
But you know you better than I do.
Baby, I'm scared to see the end.
(Knew I was made of lead).
In the clear yet? I'm nowhere near as bright.
You made me up.
You got me with a straight face.
Rock 'em (yeah!).
But I know
My Love, It's 2 A.M.
Seems like we can't have nice things, can we?.
It's 2 A.M.
All I know is a simple name, everything has changed.
All I know is a beautiful daydrea